# Day 14:  Space Stoichiometry
In this problem we are given a bunch of recipes. Each recipe shows how to combine different ingredients to make another ingredient. The eventual source of all ingredients is `ORE`, and the ultimate product of these recipes is `FUEL`. 

We need to figure out how much `ORE` we need to produce a given amount of `FUEL`.

In [128]:
from collections import defaultdict
from queue import Queue
from math import ceil

First let's write some functions for parsing the input to create our master dict of recipes.

In [129]:
def build_ingredient(string):
    parts = string.split(" ")
    return {"ingredient": parts[1], "amount": int(parts[0])}

In [130]:
def build_recipes(data):
    recipes = {}
    for line in data:
        input_str, output_str = line.split(" => ")
        
        ingredients = []
        for ingredient_str in input_str.split(', '):
            ingredients.append(build_ingredient(ingredient_str))
            
        output = build_ingredient(output_str)
        
        recipes[output["ingredient"]] = {
            "servings": output["amount"],
            "ingredients": ingredients
        }
    return recipes

Now comes the interesting part: traversing the recipes from `FUEL` to `ORE` to figure out how much `ORE` we ultimately need.

To do this, we will use a `Queue`. We'll call this `orders`, and think of it as orders in a restaurant kitchen. The first `order` is for `amount` units of `FUEL`.

We'll pick an `order` off the `orders` queue and try to fill it using the `supply` we have on hand (made up of leftovers from previous `orders`). If we don't have enough `supply`, we'll make the rest from scratch using our recipe for the ingredient in question. To do this, we figure out how much of each ingredient we need, and put each of those requests into the `orders` queue.

When the `orders` queue is empty, we know we've created all the ingredients needed for our `FUEL`, and we return the total amount of `ORE` we had to consume in the process.

In [131]:
def make_fuel(amount, recipes):
    supply = defaultdict(int)
    orders = Queue()
    orders.put({"ingredient": "FUEL", "amount": amount})
    ore_needed = 0

    while not orders.empty():
        order = orders.get()
        if order["ingredient"] == "ORE":
            ore_needed += order["amount"]
        elif order["amount"] <= supply[order["ingredient"]]:
            supply[order["ingredient"]] -= order["amount"]
        else:
            amount_needed = order["amount"] - supply[order["ingredient"]]
            recipe = recipes[order["ingredient"]]
            batches = ceil(amount_needed / recipe["servings"])
            for ingredient in recipe["ingredients"]:
                orders.put({"ingredient": ingredient["ingredient"], "amount": ingredient["amount"] * batches})
            leftover_amount = batches * recipe["servings"] - amount_needed
            supply[order["ingredient"]] = leftover_amount
    return ore_needed

Here are some test data sets to try out

In [132]:
data = [
    "10 ORE => 10 A",
    "1 ORE => 1 B",
    "7 A, 1 B => 1 C",
    "7 A, 1 C => 1 D",
    "7 A, 1 D => 1 E",
    "7 A, 1 E => 1 FUEL",
]

In [133]:
data = [
    "9 ORE => 2 A",
    "8 ORE => 3 B",
    "7 ORE => 5 C",
    "3 A, 4 B => 1 AB",
    "5 B, 7 C => 1 BC",
    "4 C, 1 A => 1 CA",
    "2 AB, 3 BC, 4 CA => 1 FUEL",
]

In [134]:
data = [
    "157 ORE => 5 NZVS",
    "165 ORE => 6 DCFZ",
    "44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL",
    "12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ",
    "179 ORE => 7 PSHF",
    "177 ORE => 5 HKGWZ",
    "7 DCFZ, 7 PSHF => 2 XJWVT",
    "165 ORE => 2 GPVTF",
    "3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT",
]

Here is the real data set.

In [135]:
with open('day_14.txt') as f:
    data = [row.strip() for row in f.readlines()]

## Part 1
For this part, we just need to make 1 unit of `FUEL`. Our `make_fuel` function will tell us how much `ORE` is needed for this.

In [136]:
recipes = build_recipes(data)

In [137]:
make_fuel(1, recipes)

365768

## Part 2
For the second part, we have to figure out how much `FUEL` we can produce with a starting supply of 1 trillion units of `ORE`. 

To do this, we'll just guess and check. We first guess how much `FUEL` we think we can produce with our huge `ORE` supply. If the ore needed for our guess is too big (over our capacity) or too small (too much `ORE` leftover), then we'll adjust and guess again. 

To do this efficiently we'll keep track of upper and lower bounds as we go, proceeding in a binary search pattern until we narrow these bounds down to only 1 possible answer.

In [138]:
upper_bound = None
lower_bound = 1
ore_capacity = 1000000000000

In [139]:
recipes = build_recipes(data)
while lower_bound + 1 != upper_bound:
    if upper_bound is None:
        guess = lower_bound * 2
    else:
        guess = (upper_bound + lower_bound) // 2
        
    ore_needed = make_fuel(guess, recipes)
    if ore_needed > ore_capacity:
        upper_bound = guess
    else:
        lower_bound = guess

In [140]:
lower_bound

3756877